In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
%reload_ext autoreload

In [19]:
import pandas as pd
from datetime import datetime, timedelta
import re
import sys
import os
import numpy as np

In [20]:
sys.path.append(os.path.abspath('../scripts/'))
sys.path.append(os.path.abspath('../src/'))

In [21]:
from data_loader import load_data
from vendor_board.vendor_analytics_board import extract_entities

In [22]:
# Load the CSV file
df = load_data('../data/cleaned_message.csv')

In [23]:
df.head()

,Unnamed: 0,Channel Title,Channel Username,Message ID,Sender,Date,Message,Media_Path
0,0,Zemen Express®,@ZemenExpress,6982,NaN,2025-06-18 06:01:10+00:00,በኤሌክትሪክ:የሚሰራ ለቤት መልካም መዓዛን የሚሰጥ ዋጋ፦ 1400 ብር ውስ...,[ምስል የለም]
1,1,Zemen Express®,@ZemenExpress,6981,NaN,2025-06-16 12:21:00+00:00,በፈለጉት አቅጣጫ ልጅዎን በምቾት ማዘል ያስችልዎታል ዋጋ፦ 2400 ብር ው...,[ምስል የለም]
2,2,Zemen Express®,@ZemenExpress,6980,NaN,2025-06-16 05:11:57+00:00,መልዕክት የለም,data\photos\@ZemenExpress_6980.jpg
3,3,Zemen Express®,@ZemenExpress,6979,NaN,2025-06-16 05:11:57+00:00,መልዕክት የለም,data\photos\@ZemenExpress_6979.jpg
4,4,Zemen Express®,@ZemenExpress,6978,NaN,2025-06-16 05:11:57+00:00,መልዕክት የለም,data\photos\@ZemenExpress_6978.jpg


In [ ]:
# Preprocess data
df["Date"] = pd.to_datetime(df["Date"])
df = df[df["Message"].notnull() & (df["Message"] != "")]  # Remove empty messages
df = df[~df["Message"].str.contains("ሱቅ መተው|እንኳን ለኢድ")]  # Exclude non-product posts

np.random.seed(42)
df["Views"] = np.random.randint(100, 1000, size=len(df))

In [25]:
# Apply NER to extract prices and products
df[["Price", "Product"]] = df["Message"].apply(lambda x: pd.Series(extract_entities(x)))

# Filter posts with valid prices
df = df[df["Price"].notnull()]

# Define analysis period
start_date = df["Date"].min()
end_date = df["Date"].max()
weeks = (end_date - start_date).days / 7 if (end_date - start_date).days > 0 else 1

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8979 entries, 0 to 36328
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   Unnamed: 0        8979 non-null   int64              
 1   Channel Title     8979 non-null   object             
 2   Channel Username  8979 non-null   object             
 3   Message ID        8979 non-null   int64              
 4   Sender            0 non-null      float64            
 5   Date              8979 non-null   datetime64[ns, UTC]
 6   Message           8979 non-null   object             
 7   Media_Path        8979 non-null   object             
 8   Views             8979 non-null   int32              
 9   Price             8979 non-null   float64            
 10  Product           8617 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(2), int32(1), int64(2), object(5)
memory usage: 806.7+ KB


In [27]:
# Initialize results
vendor_metrics = []

# Process each vendor
for vendor in df["Channel Username"].unique():
    vendor_data = df[df["Channel Username"] == vendor]
    
    # Posting Frequency: Posts per week
    posts_count = len(vendor_data)
    posting_frequency = posts_count / weeks
    
    # Average Views per Post
    avg_views = vendor_data["Views"].mean()
    
    # Top Performing Post
    top_post = vendor_data.loc[vendor_data["Views"].idxmax()]
    top_post_views = top_post["Views"]
    top_post_product = top_post["Product"] if pd.notnull(top_post["Product"]) else "Unknown"
    top_post_price = top_post["Price"] if pd.notnull(top_post["Price"]) else 0
    top_post_id = top_post["Message ID"]
    
    # Average Price Point
    avg_price = vendor_data["Price"].mean()
    
    # Lending Score: Normalize views and frequency
    max_views = df["Views"].max()
    max_frequency = df.groupby("Channel Username").size().max() / weeks
    normalized_views = avg_views / max_views if max_views > 0 else 0
    normalized_frequency = posting_frequency / max_frequency if max_frequency > 0 else 0
    lending_score = (normalized_views * 0.5) + (normalized_frequency * 0.5)
    
    # Store metrics
    vendor_metrics.append({
        "Vendor": vendor,
        "Avg Views/Post": round(avg_views, 2),
        "Posts/Week": round(posting_frequency, 2),
        "Avg Price (ETB)": round(avg_price, 2),
        "Lending Score": round(lending_score, 2),
        "Top Post Product": top_post_product,
        "Top Post Price (ETB)": top_post_price,
        "Top Post Views": top_post_views,
        "Top Post ID": top_post_id
    })

In [28]:
# Create results DataFrame
results_df = pd.DataFrame(vendor_metrics)

# Generate Vendor Scorecard as markdown table
scorecard = "# Vendor Scorecard\n\n"
scorecard += "| Vendor | Avg Views/Post | Posts/Week | Avg Price (ETB) | Lending Score |\n"
scorecard += "|--------|----------------|------------|-----------------|---------------|\n"
for _, row in results_df.iterrows():
    scorecard += f"| {row['Vendor']} | {row['Avg Views/Post']} | {row['Posts/Week']} | {row['Avg Price (ETB)']} | {row['Lending Score']} |\n"

# Save scorecard to file
with open("../data/vendor_scorecard.md", "w", encoding="utf-8") as f:
    f.write(scorecard)

# Print top post details
print("Top Performing Posts:")
for _, row in results_df.iterrows():
    print(f"{row['Vendor']}: Product = {row['Top Post Product']}, Price = {row['Top Post Price (ETB)']} ETB, Views = {row['Top Post Views']}, Message ID = {row['Top Post ID']}")

# Save full results to CSV
results_df.to_csv("../data/vendor_metrics.csv", index=False)

Top Performing Posts:
@ZemenExpress: Product = : 385ml, Price = 450.0 ETB, Views = 999, Message ID = 6779
@nevacomputer: Product = የብረቱ ላፕቶፕ ጭን ላይ አስቀምጠው መጠቀም ለከፋ ጉዳት እንደሚያጋልጥ የጤና ባለሙያዎች ይናገራሉ ፤ እንዲሁም ቴብል ላይ አድርገዉ ሲጠቀሙ ፋኑ ስለሚደፈን ከፍተኛ ሙቀት በመፍጠር ድምፅ ያሰማል። ለዚህ ሁሉ መፍትሄ ተጣጣፊ የላፕቶፕ ፣ ታብሌት በተመጣጣኝ ዋጋ ገዝተዉ የእርስዎንና የላፕቶፕዎን ጤንነት ይጠብቁ።, Price = 1300.0 ETB, Views = 961, Message ID = 6568
@qnashcom: Product = 50mm 15, Price = 650.0 ETB, Views = 997, Message ID = 2536
@gebeyaadama: Product = ለተሰነጣጠቀ ለደረቅ ተረከዝ ሞረድ እና ማለስለሻ, Price = 750.0 ETB, Views = 998, Message ID = 3188
@Shageronlinestore: Product = , Price = 200.0 ETB, Views = 999, Message ID = 3725
